In [1]:
import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd
import json
import os
from tps import ThinPlateSpline
from skspatial.objects import Plane, Points

In [2]:
import plotly.graph_objects as go
import plotly.express as px

import pyvista as pv
# pip install --upgrade trame-vtk
pv.set_jupyter_backend('trame')

In [3]:
from rotation_utils import *

In [4]:
os.getcwd()

'd:\\Docs_ASUS\\WORK\\hip\\MotionStudy\\MotionData\\003\\P03'

<div class="alert alert-block alert-info">
<b>REMEMBER:</b> OSIM works in meters, 3DSlicer works in milimeters - watch scaling of data!
</div>

## Import OSIM Markers from XML file

In [5]:
# parse the .xml with markers in body frames
markers_tree=ET.parse("markers_and_bone_markers_in_bodies.xml")
markers_root = markers_tree.getroot()
print(markers_root)

<Element 'OpenSimDocument' at 0x0000013AEF5C7DB0>


In [6]:
# extract markers and create a list of dictionaries
mrkrs = []
for Marker in markers_root.iter('Marker'):
    dic = Marker.attrib
    for child in Marker:
        tagg = child.tag
        dic[tagg] = child.text
    mrkrs.append(dic)
len(mrkrs)

108

In [ ]:
# split information in preparation for data frame
mrkrs_new = []
keys = ['name', 'socket_parent_frame', 'location']
for mrkr in mrkrs:
    dic = {}
    for key in keys:
        if key == 'name':
            dic[key] = mrkr[key]
        if key == 'socket_parent_frame':
            if '/bodyset' in mrkr[key]:
                mrkr[key] = mrkr[key][9:]
            elif '/' in mrkr[key]:
                mrkr[key] = mrkr[key][1:]
            dic[key] = mrkr[key]
        if key == 'location':
            a = mrkr[key].split()
            a = [float(i) for i in a]
            dic['r'] = a[0]*1000
            dic['a'] = a[1]*1000
            dic['s'] = a[2]*1000
    mrkrs_new.append(dic)

In [ ]:
# create the dataframe that contains all information
df_markers = pd.DataFrame.from_dict(mrkrs_new)
# df_markers.head()
# export the dataframe to CVS
# df_markers.to_csv('model_update/markers_in_child.csv')

##  Import OSIM muscles and muscle wrapping surfaces from the Rajagopal model file

In [ ]:
# Import and parse the original Rajagopal model with muscles
tree_model = ET.parse('RajagopalModified_generic_copy.osim')
root_model = tree_model.getroot()

#### Import muscles

In [ ]:
# Extract the muscle path points
lst = []
for PathPointSet in root_model.iter('PathPointSet'):
    lst.append(PathPointSet)

lm_groups = []
for N in lst:
    n = N[0]
    l = []
    for el in n:
        dic = el.attrib
        for child in el:
            tagg = child.tag
            dic[tagg] = child.text
        l.append(dic)
    lm_groups.append(l)

In [ ]:
# Split information in preparation for a dataframe
m_data = []
for muscle in lm_groups:
    for landmark in muscle:
        entry = {}
        lm = landmark['name']
        entry['label'] = lm

        msk = lm[:-3]
        entry['muscle'] = msk
  
        a = landmark['location'].split()
        a = [float(i) for i in a]
        entry['r'] = a[0]*1000
        entry['a'] = a[1]*1000
        entry['s'] = a[2]*1000
        
        body = landmark['socket_parent_frame']
        entry['description'] = body[9:]        
        m_data.append(entry)

In [ ]:
# create the dataframe with all muscles and their bodies
df_muscles = pd.DataFrame(m_data)
# df_muscles.head()

#### Import muscle wrapping objects

<div class="alert alert-block alert-info"> <b>NOTE</b> The output dataframe does not split translation and rotation into axes. </div>

In [ ]:
# Extract the muscle path points
wrp_lst = []
for Body in root_model.iter('Body'):
    body_name = Body.get('name')
    objs = Body.iter('WrapCylinder')
    # print(objs)
    for obj in objs:
        ob = {}
        obj_name = obj.get('name')
        rotation = (obj.find('xyz_body_rotation').text).split()
        rotation = np.array([float(i) for i in rotation])
        translation = (obj.find('translation').text).split()
        translation = np.array([float(i) for i in translation])
        radius = float(obj.find('radius').text)
        half_length = 0.5*float(obj.find('length').text)
        ob['body'], ob['name'] = (body_name, obj_name)
        ob['rotation'], ob['translation'] = (rotation, translation*1000) # rotation is in radians
        ob['radius'], ob['half_length'] = (radius*1000, half_length*1000)
        wrp_lst.append(ob)

wrp_df = pd.DataFrame(wrp_lst)
wrp_df.tail(10)

In [ ]:
wrp_df['radius_point']=['']*wrp_df.shape[0]
wrp_df['axis_point']=['']*wrp_df.shape[0]

In [ ]:
for i, row in wrp_df.iterrows():
    axes = np.array([[1,0,0],[0,1,0],[0,0,1]])
    angles = row['rotation']
    center = row['translation']

    x_rotation=rotation_matrix(axes[0], angles[0])
    y_rotation=rotation_matrix(axes[1], angles[1])
    z_rotation=rotation_matrix(axes[2], angles[2])
    R = np.matmul(z_rotation, x_rotation, y_rotation)

    radius_point = np.matmul(R,axes[0])*row['radius'] + center
    axis_point = np.matmul(R,axes[2])*row['half_length']*0.5 + center
    # print(radius_point)
    wrp_df._set_value(i,'radius_point', radius_point) # = 
    wrp_df._set_value(i,'axis_point', axis_point)

In [ ]:
body_lst = wrp_df['body'].unique()
for body in body_lst:
    temp = []
    temp.append((wrp_df[wrp_df['body']==body])[['name', 'translation','radius_point', 'axis_point']])
    globals()[f'{body}_wrp'] = pd.concat(temp)

In [ ]:
wrapping_surface_markers = [pelvis_wrp, femur_r_wrp, femur_l_wrp, tibia_r_wrp, tibia_l_wrp]

## Import Markers from Sclicer3D file

In [ ]:
# create a function for parsing info from a json file
## !! ATTENTION: !! 3DSlicer json separates the position and orientation in two tensors. To obtain the Viewer's orientation one needs to multiply them.
def load_orient_json(path_n_file, orient = True):
    file = open(path_n_file)
    data = json.load(file)
    points = {}
    for item in data['markups'][0]['controlPoints']:
        orient = np.array(item['orientation'])
        orient.shape = (3,3)
        position = np.matmul(orient, np.array(item['position']))
        points[item['label']] = position.tolist()
    return points

In [ ]:
path = os.path.abspath("../MRI/003-Anonymous/P03")
mri_bone_markers = load_orient_json(os.path.join(path, "orientation.mrk.json"), orient = True)

## Match bone markers for TPS function

In [ ]:
# match names of markers in 3DSlicer file with OSIM model
rows = []
not_in_mri = []
for key in df_markers['name']:
    if key in mri_bone_markers.keys():
        rows.append(df_markers[df_markers['name']==key])
    else: not_in_mri.append(df_markers[df_markers['name']==key])
osim_bone_markers_df = pd.concat(rows)
osim_not_in_mri_markers_df = pd.concat(not_in_mri)

In [ ]:
# the support markers were created to control the position of joint centres in parent bodies
# currently, they are not in the list of MRI markers

osim_support_bone_markers_df = osim_not_in_mri_markers_df.iloc[-9:]
print(osim_support_bone_markers_df['name'])

In [ ]:
print(osim_bone_markers_df.shape[0])

In [ ]:
bodies = osim_bone_markers_df['socket_parent_frame'].unique()
for body in bodies:
    globals()[f'{body}_list'] = []
    globals()[f'{body}_list'].append(osim_bone_markers_df[osim_bone_markers_df['socket_parent_frame']==body])

In [ ]:
# create dataframes for bone markers in separate bodies
pelvis_osim = pd.concat(pelvis_list)[['name', 'r', 'a', 's']]
femur_r_osim = pd.concat(femur_r_list)[['name', 'r', 'a', 's']]
femur_l_osim = pd.concat(femur_l_list)[['name', 'r', 'a', 's']]
tibia_r_osim = pd.concat(tibia_r_list)[['name', 'r', 'a', 's']]
tibia_l_osim = pd.concat(tibia_l_list)[['name', 'r', 'a', 's']]
patella_r_osim = pd.concat(patella_r_list)[['name', 'r', 'a', 's']]
patella_l_osim = pd.concat(patella_l_list)[['name', 'r', 'a', 's']]

In [ ]:
# create a function for extracting muscle paths
def extract_data(df, body):
 rows = []
 rows.append(df[df['description']==body])
 df_rows = pd.concat(rows)[['label', 'r', 'a', 's']]
 df_new = df_rows.rename(columns={'label':'name',})
 return df_new

In [ ]:
# muscles alone
pelvis_osim_mscl = extract_data(df_muscles, 'pelvis')
femur_r_osim_mscl = extract_data(df_muscles, 'femur_r')
femur_l_osim_mscl = extract_data(df_muscles, 'femur_l')
tibia_r_osim_mscl = extract_data(df_muscles, 'tibia_r')
tibia_l_osim_mscl = extract_data(df_muscles, 'tibia_l')
patella_r_osim_mscl = extract_data(df_muscles, 'patella_r')
patella_l_osim_mscl = extract_data(df_muscles, 'patella_l')

In [ ]:
#osim_support_bone_markers_df

In [ ]:
pelvis_support = osim_support_bone_markers_df[osim_support_bone_markers_df['socket_parent_frame']=='pelvis'][['name', 'r', 'a', 's']]
femur_r_support = osim_support_bone_markers_df[osim_support_bone_markers_df['socket_parent_frame']=='femur_r'][['name', 'r', 'a', 's']]
femur_l_support = osim_support_bone_markers_df[osim_support_bone_markers_df['socket_parent_frame']=='femur_l'][['name', 'r', 'a', 's']]
tibia_r_support = osim_support_bone_markers_df[osim_support_bone_markers_df['socket_parent_frame']=='tibia_r'][['name', 'r', 'a', 's']]
tibia_l_support = osim_support_bone_markers_df[osim_support_bone_markers_df['socket_parent_frame']=='tibia_l'][['name', 'r', 'a', 's']]
#patella_r_support = osim_support_bone_markers_df[osim_support_bone_markers_df['socket_parent_frame']=='patella_r'][['name', 'r', 'a', 's']]
#patella_r_support = osim_support_bone_markers_df[osim_support_bone_markers_df['socket_parent_frame']=='patella_l'][['name', 'r', 'a', 's']]

In [ ]:
tibia_l_support 

In [ ]:
# bone markers and muscles together
pelvis_osim_mark_mscl = pd.concat([pelvis_osim, pelvis_support, pelvis_osim_mscl], ignore_index=True)
femur_r_osim_mark_mscl = pd.concat([femur_r_osim, femur_r_support,  femur_r_osim_mscl], ignore_index=True)
femur_l_osim_mark_mscl = pd.concat([femur_l_osim, femur_l_support, femur_l_osim_mscl], ignore_index=True)
tibia_r_osim_mark_mscl = pd.concat([tibia_r_osim, tibia_r_support,  tibia_r_osim_mscl], ignore_index=True)
tibia_l_osim_mark_mscl = pd.concat([tibia_l_osim, tibia_l_support, tibia_l_osim_mscl], ignore_index=True)
patella_r_osim_mark_mscl = pd.concat([patella_r_osim, patella_r_osim_mscl], ignore_index=True)
patella_l_osim_mark_mscl = pd.concat([patella_l_osim, patella_l_osim_mscl], ignore_index=True)

## Separate 3DSlicer bone markers into bodies

In [ ]:
# create a function for splitting the dataframe into bodies
def choose_lms(dict_target, df_template):
    names = df_template['name']
    rows = []
    for name in names:
        rows.append({'name' : name, 'r' : dict_target[name][0], 'a' : dict_target[name][1], 's' : dict_target[name][2]})
    return pd.DataFrame.from_dict(rows)

In [ ]:
pelvis_mri = choose_lms(mri_bone_markers, pelvis_osim)
femur_r_mri = choose_lms(mri_bone_markers, femur_r_osim)
femur_l_mri = choose_lms(mri_bone_markers, femur_l_osim)
tibia_r_mri = choose_lms(mri_bone_markers, tibia_r_osim)
tibia_l_mri = choose_lms(mri_bone_markers, tibia_l_osim)
patella_r_mri = choose_lms(mri_bone_markers, patella_r_osim)
patella_l_mri = choose_lms(mri_bone_markers, patella_l_osim)

## TPS markers and muscles

<div class="alert alert-block alert-info"> <b>NOTE</b> The transformed features are returned in mm. </div>

#### _pelvis_

In [ ]:
tps_pelvis = ThinPlateSpline(alpha = 0)
tps_pelvis.fit(pelvis_osim[['r','a','s']].to_numpy(), pelvis_mri[['r','a','s']].to_numpy())
pelvis_result = tps_pelvis.transform(pelvis_osim_mark_mscl[['r','a','s']].to_numpy())

pelvis_mrkrs_mscls_names = list(pelvis_osim_mark_mscl['name'])


### _femora_

In [ ]:
tps_femur_r = ThinPlateSpline(alpha = 0)
tps_femur_r.fit(femur_r_osim[['r','a','s']].to_numpy(), femur_r_mri[['r','a','s']].to_numpy())
femur_r_result = tps_femur_r.transform(femur_r_osim_mark_mscl[['r','a','s']].to_numpy())

femur_r_mrkrs_mscls_names = list(femur_r_osim_mark_mscl['name'])


In [ ]:
tps_femur_l = ThinPlateSpline(alpha = 0)
tps_femur_l.fit(femur_l_osim[['r','a','s']].to_numpy(), femur_l_mri[['r','a','s']].to_numpy())
femur_l_result = tps_femur_l.transform(femur_l_osim_mark_mscl[['r','a','s']].to_numpy())

femur_l_mrkrs_mscls_names = list(femur_l_osim_mark_mscl['name'])


### _patellas_

In [ ]:
tps_patella_r = ThinPlateSpline(alpha = 0)
tps_patella_r.fit(patella_r_osim[['r','a','s']].to_numpy(), patella_r_mri[['r','a','s']].to_numpy())
patella_r_result = tps_patella_r.transform(patella_r_osim_mark_mscl[['r','a','s']].to_numpy())

patella_r_mrkrs_mscls_names = list(patella_r_osim_mark_mscl['name'])


In [ ]:
tps_patella_l = ThinPlateSpline(alpha = 0)
tps_patella_l.fit(patella_l_osim[['r','a','s']].to_numpy(), patella_l_mri[['r','a','s']].to_numpy())
patella_l_result = tps_patella_l.transform(patella_l_osim_mark_mscl[['r','a','s']].to_numpy())

patella_l_mrkrs_mscls_names = list(patella_l_osim_mark_mscl['name'])


### _tibias_

In [ ]:
tps_tibia_r = ThinPlateSpline(alpha = 0)
tps_tibia_r.fit(tibia_r_osim[['r','a','s']].to_numpy(), tibia_r_mri[['r','a','s']].to_numpy())
tibia_r_result = tps_tibia_r.transform(tibia_r_osim_mark_mscl[['r','a','s']].to_numpy())

tibia_r_mrkrs_mscls_names = list(tibia_r_osim_mark_mscl['name'])


In [ ]:
tps_tibia_l = ThinPlateSpline(alpha = 0)
tps_tibia_l.fit(tibia_l_osim[['r','a','s']].to_numpy(), tibia_l_mri[['r','a','s']].to_numpy())
tibia_l_result = tps_tibia_l.transform(tibia_l_osim_mark_mscl[['r','a','s']].to_numpy())

tibia_l_mrkrs_mscls_names = list(tibia_l_osim_mark_mscl['name'])


## TPS muscle wrapping surfaces

#### _pelvis_

In [ ]:
pl_n = list(pelvis_wrp['name'])
pl_r = tps_pelvis.transform(np.array([i for i in pelvis_wrp['radius_point']]))
pl_a = tps_pelvis.transform(np.array([i for i in pelvis_wrp['axis_point']]))
pl_t = tps_pelvis.transform(np.array([i for i in pelvis_wrp['translation']]))

#### _femur r_

In [ ]:
fr_n = list(femur_r_wrp['name'])
fr_r = tps_femur_r.transform(np.array([i for i in femur_r_wrp['radius_point']]))
fr_a = tps_femur_r.transform(np.array([i for i in femur_r_wrp['axis_point']]))
fr_t = tps_femur_r.transform(np.array([i for i in femur_r_wrp['translation']]))

#### _femur l_

In [ ]:
fl_n = list(femur_l_wrp['name'])
fl_r = tps_femur_l.transform(np.array([i for i in femur_l_wrp['radius_point']]))
fl_a = tps_femur_l.transform(np.array([i for i in femur_l_wrp['axis_point']]))
fl_t = tps_femur_l.transform(np.array([i for i in femur_l_wrp['translation']]))

#### _tibia r_

In [ ]:
tr_n = list(tibia_r_wrp['name'])
tr_r = tps_tibia_r.transform(np.array([i for i in tibia_r_wrp['radius_point']]))
tr_a = tps_tibia_r.transform(np.array([i for i in tibia_r_wrp['axis_point']]))
tr_t = tps_tibia_r.transform(np.array([i for i in tibia_r_wrp['translation']]))

#### _tibia l_

In [ ]:
tl_n = list(tibia_l_wrp['name'])
tl_r = tps_tibia_l.transform(np.array([i for i in tibia_l_wrp['radius_point']]))
tl_a = tps_tibia_l.transform(np.array([i for i in tibia_l_wrp['axis_point']]))
tl_t = tps_tibia_l.transform(np.array([i for i in tibia_l_wrp['translation']]))

## Import and TPS surfaces

#### pelvis

In [ ]:
pl_mesh = pv.read('Geometry/l_pelvis.vtp')
pl_faces = pl_mesh.faces
pr_mesh = pv.read('Geometry/r_pelvis.vtp')
pr_faces = pr_mesh.faces
sacrum_mesh = pv.read('Geometry/sacrum.vtp')
sacrum_faces = sacrum_mesh.faces

In [ ]:
pl_pts = tps_pelvis.transform(np.array(pl_mesh.points)*1000)
pr_pts = tps_pelvis.transform(np.array(pr_mesh.points)*1000)
sacrum_pts = tps_pelvis.transform(np.array(sacrum_mesh.points)*1000)

#### femur_r

In [ ]:
fr_mesh = pv.read('Geometry/r_femur.vtp')
fr_pts = tps_femur_r.transform(np.array(fr_mesh.points)*1000)
fr_faces = fr_mesh.faces

In [ ]:
the_pts_transformed = fr_pts/1000
new_mesh = pv.PolyData(the_pts_transformed, fr_mesh.faces)
#new_mesh.save('model_update/femur_r.stl')
plotter = pv.Plotter(window_size=(600, 400))

plotter.background_color = 'w'
plotter.enable_anti_aliasing()
plotter.add_mesh(fr_mesh, color='lightblue', show_edges=True)
plotter.add_mesh(new_mesh, color='red', show_edges=True)
plotter.show(jupyter_backend='trame')

#### femur_l

In [ ]:
fl_mesh = pv.read('Geometry/l_femur.vtp')
fl_pts = tps_femur_l.transform(np.array(fl_mesh.points)*1000)
fl_faces = fl_mesh.faces

#### tibia_r

In [ ]:
tr_mesh = pv.read('Geometry/r_tibia.vtp')
tr_pts = tps_tibia_r.transform(np.array(tr_mesh.points)*1000)
tr_faces = tr_mesh.faces

fib_r_mesh = pv.read('Geometry/r_fibula.vtp')
fib_r_pts = tps_tibia_r.transform(np.array(fib_r_mesh.points)*1000)
fib_r_faces = fib_r_mesh.faces

In [ ]:
the_pts_transformed = tr_pts/1000
new_mesh = pv.PolyData(the_pts_transformed, tr_faces)
#new_mesh.save('model_update/femur_r.stl')
plotter = pv.Plotter(window_size=(600, 400))

plotter.background_color = 'w'
plotter.enable_anti_aliasing()
plotter.add_mesh(tr_mesh, color='lightblue', show_edges=True)
plotter.add_mesh(new_mesh, color='red', show_edges=True)
plotter.show(jupyter_backend='trame')

#### tibia_l

In [ ]:
tl_mesh = pv.read('Geometry/l_tibia.vtp')
tl_pts = tps_tibia_l.transform(np.array(tl_mesh.points)*1000)
tl_faces = tl_mesh.faces

fib_l_mesh = pv.read('Geometry/l_fibula.vtp')
fib_l_pts = tps_tibia_l.transform(np.array(fib_l_mesh.points)*1000)
fib_l_faces = fib_l_mesh.faces

In [ ]:
the_pts_transformed = tl_pts/1000
new_mesh = pv.PolyData(the_pts_transformed, tl_faces)
#new_mesh.save('model_update/femur_r.stl')
plotter = pv.Plotter(window_size=(600, 400))

plotter.background_color = 'w'
plotter.enable_anti_aliasing()
plotter.add_mesh(tl_mesh, color='lightblue', show_edges=True)
plotter.add_mesh(new_mesh, color='red', show_edges=True)
plotter.show(jupyter_backend='trame')

#### patella_r

In [ ]:
ptlr_mesh = pv.read('Geometry/r_patella.vtp')
ptlr_pts = tps_patella_r.transform(np.array(ptlr_mesh.points)*1000)
ptlr_faces = ptlr_mesh.faces

#### patella_l

In [ ]:
ptll_mesh = pv.read('Geometry/l_patella.vtp')
ptll_pts = tps_patella_l.transform(np.array(ptll_mesh.points)*1000)
ptll_faces = ptll_mesh.faces

In [ ]:
import plotly.graph_objects as go
import plotly.express as px

fig = px.scatter()

fig.add_trace(go.Scatter3d(
x=ptlr_pts[:,0], 
y=ptlr_pts[:,1], 
z=ptlr_pts[:,2],
marker=dict(
    size=4,
    opacity=0.8
    ),
mode = 'markers',
))

fig.add_trace(go.Scatter3d(
x=patella_r_result[:,0], 
y=patella_r_result[:,1], 
z=patella_r_result[:,2],
marker=dict(
    size=4,
    opacity=0.8
),
mode = 'markers + text',
text=patella_r_osim_mark_mscl['name'],
textposition="top center"))

fig.update_scenes(aspectmode="data" )
fig.show()

## Rotate bodies to child frame

In [ ]:
def plot_transformed(points_file, order_file, mrkr_mscl_names):
   fig = px.scatter()
   for key in order_file.keys():
      if key == 'mrkrs_mscls':
         fig.add_trace(go.Scatter3d(
            x=points_file[order_file['mrkrs_mscls'],0], 
            y=points_file[order_file['mrkrs_mscls'],1], 
            z=points_file[order_file['mrkrs_mscls'],2],
            marker=dict(
               size=4,
               opacity=0.8
            ),
            mode = 'markers + text',
            text=mrkr_mscl_names,
            textposition="top center"))
      else:
         fig.add_trace(go.Scatter3d(
            x=points_file[order_file[key],0], 
            y=points_file[order_file[key],1], 
            z=points_file[order_file[key],2],
            marker=dict(
               size=4,
               opacity=0.8
            ),
            mode = 'markers',
         ))

   fig.update_scenes(aspectmode="data" )
   fig.show()

In [ ]:
# Given that 3D sclicer interprets OSIM xyt as ras (left to right, posterior to anterior, inferior to superior), an adjustment should be made in order to obtain the correct
# orientation of the imported data
# pelvis_osim =  np.array(([1,0,0], [0,1,0], [0,0,1]))
axes_swap = np.array([[0,0,1], [1,0,0], [0,1,0]])
axes_osim = axes_swap*50

In [ ]:
# check names of lms, create a table for all and extract eigenvalues to determine the major vertical axis
li = []
for key in mri_bone_markers.keys():
    li.append(mri_bone_markers[key])
eval, evec = np.linalg.eigh(np.array(li).T@np.array(li), UPLO='L')

#### _pelvis_

In [ ]:
mri_bone_markers.keys()

In [ ]:
# rotation matrix
p1 = np.array(mri_bone_markers['ASIS_r'])
p2 =(np.array(mri_bone_markers['PSIS_l']) + np.array(mri_bone_markers['PSIS_r']))*0.5
p3 = np.array(mri_bone_markers['ASIS_l'])
p4 = mri_bone_markers['pelvis_origin']
pelvis_location = p4

pelvis_PA = (p4-p2)/np.linalg.norm(p4-p2) # PA stands for posterior -> anterior
pelvis_IS = evec[:, 2]
pelvis_LR = np.cross(pelvis_PA, pelvis_IS)# LR stand s for left -> right

axes_image = (np.array((pelvis_LR, pelvis_PA, pelvis_IS)) * 50) + pelvis_location 

pelvis_R, pelvis_t, pelvis_center_osim, pelvis_center_image = axes_rotation(axes_osim, axes_image) # mri and osim are 3x3 matrices - second is transferred to the first

In [ ]:
# assemble data for transformation and keep record of the order
obj_order = ['mrkrs_mscls', 'wrp_rad', 'wrp_ax', 'wrp_transl', 'pelv_l_pts', 'pelv_r_pts', 'sacr_pts']
pelvis_lm_order = {}
n = 0
for i, num in enumerate([pelvis_result.shape[0], pl_r.shape[0], pl_a.shape[0], pl_t.shape[0], pl_pts.shape[0], pr_pts.shape[0], sacrum_pts.shape[0]]):
    pelvis_lm_order[obj_order[i]] = list(range(n, n+num))
    n += num

pelvis_to_transform = np.concatenate([pelvis_result, pl_r, pl_a, pl_t, pl_pts, pr_pts, sacrum_pts])

In [ ]:
# rotate and shift origin to 0,0,0
pelvis_rotated = apply_rotate_zero_origin(pelvis_to_transform, pelvis_R)
pelvis_origin = np.mean(pelvis_rotated[:2], axis = 0) # mean of ASIS potins
pelvis_transformed = pelvis_rotated - pelvis_origin

In [ ]:
# scale surface for OpenSim and save
l_pelvis_transformed = pv.PolyData(pelvis_transformed[pelvis_lm_order['pelv_l_pts']]/1000, pl_faces)
r_pelvis_transformed = pv.PolyData(pelvis_transformed[pelvis_lm_order['pelv_r_pts']]/1000, pr_faces)
sacrum_transformed = pv.PolyData(pelvis_transformed[pelvis_lm_order['sacr_pts']]/1000, sacrum_faces)

l_pelvis_transformed.save('model_update/tps_warping_results/l_pelvis.stl')
r_pelvis_transformed.save('model_update/tps_warping_results/r_pelvis.stl')
sacrum_transformed.save('model_update/tps_warping_results/sacrum.stl')

In [ ]:
# scale markers and muscles, create a dataframe for sorting and saving
body = 'pelvis'
pelvis_transformed_mrkr_mscl = {'body':[], 'name':[], 'location':[]}
for i, n in enumerate(pelvis_mrkrs_mscls_names):
    pelvis_transformed_mrkr_mscl['body'].append('pelvis'),
    pelvis_transformed_mrkr_mscl['name'].append(n),
    pelvis_transformed_mrkr_mscl['location'].append(pelvis_transformed[pelvis_lm_order['mrkrs_mscls']][i]/1000)
pelvis_transformed_mrkr_mscl_df = pd.DataFrame(pelvis_transformed_mrkr_mscl)

#### *femur_r*

In [ ]:
# rotation matrix
femur_r_location = np.array(mri_bone_markers['femur_r_center'])
points = ['femur_r_center', 'knee_r_med', 'knee_r_lat']

femur_r_pa = find_axis(mri_bone_markers, points)
femur_r_is = (np.array(mri_bone_markers['femur_r_center']) - np.array(mri_bone_markers['knee_r_center']))/np.linalg.norm(np.array(mri_bone_markers['femur_r_center']) - np.array(mri_bone_markers['knee_r_center']))
femur_r_lr = np.cross(femur_r_pa, femur_r_is)/np.linalg.norm(np.cross(femur_r_pa, femur_r_is))

femur_r_axes = np.array((femur_r_lr, femur_r_pa, femur_r_is)) * 50 + femur_r_location

femur_r_R, femur_r_t, femur_r_center_osim, femur_r_center_mri = axes_rotation(axes_osim, femur_r_axes)

In [ ]:
# assemble data for transformation and keep record of the order
obj_order = ['mrkrs_mscls', 'wrp_rad', 'wrp_ax', 'wrp_transl', 'surf_pts']
femur_r_lm_order = {}
n = 0
for i, num in enumerate([femur_r_result.shape[0], fr_r.shape[0], fr_a.shape[0], fr_t.shape[0], fr_pts.shape[0]]):
    femur_r_lm_order[obj_order[i]] = list(range(n, n+num))
    n += num

femur_r_to_transform = np.concatenate([femur_r_result, fr_r, fr_a, fr_t, fr_pts])

In [ ]:
# rotate and shift origin to 0,0,0
femur_r_rotated = apply_rotate_zero_origin(femur_r_to_transform, femur_r_R)
femur_r_origin = femur_r_rotated[2] # femur head centre
femur_r_transformed = femur_r_rotated - femur_r_origin

In [ ]:
# scale surface for OpenSim and save
the_pts_transformed = femur_r_transformed[femur_r_lm_order['surf_pts']]/1000
new_mesh = pv.PolyData(the_pts_transformed, fr_faces)
new_mesh.save('model_update/tps_warping_results/femur_r.stl')

In [ ]:
# scale markers and muscles, create a dataframe for sorting and saving
body = 'femur_r'
femur_r_transformed_mrkr_mscl = {'body':[], 'name':[], 'location':[]}
for i, n in enumerate(femur_r_mrkrs_mscls_names):
    femur_r_transformed_mrkr_mscl['body'].append('femur_r'),
    femur_r_transformed_mrkr_mscl['name'].append(n),
    femur_r_transformed_mrkr_mscl['location'].append(femur_r_transformed[femur_r_lm_order['mrkrs_mscls']][i]/1000)
femur_r_transformed_mrkr_mscl_df = pd.DataFrame(femur_r_transformed_mrkr_mscl)

#### *femur_l*

In [ ]:
# rotation matrix
femur_l_location = np.array(mri_bone_markers['femur_l_center'])
points = ['femur_l_center', 'knee_l_lat', 'knee_l_med']

femur_l_pa = find_axis(mri_bone_markers, points)
femur_l_is = (np.array(mri_bone_markers['femur_l_center']) - np.array(mri_bone_markers['knee_l_center']))/np.linalg.norm(np.array(mri_bone_markers['femur_l_center']) - np.array(mri_bone_markers['knee_l_center']))
femur_l_lr = np.cross(femur_l_pa, femur_r_is)/np.linalg.norm(np.cross(femur_l_pa, femur_l_is))

femur_l_axes = np.array((femur_l_lr, femur_l_pa, femur_l_is)) * 50 + femur_l_location

femur_l_R, femur_l_t, femur_l_center_osim, femur_l_center_mri = axes_rotation(axes_osim, femur_l_axes)

In [ ]:
# assemble data for transformation and keep record of the order
obj_order = ['mrkrs_mscls', 'wrp_rad', 'wrp_ax', 'wrp_transl', 'surf_pts']
femur_l_lm_order = {}
n = 0
for i, num in enumerate([femur_l_result.shape[0], fl_r.shape[0], fl_a.shape[0], fl_t.shape[0], fl_pts.shape[0]]):
    femur_l_lm_order[obj_order[i]] = list(range(n, n+num))
    n += num

femur_l_to_transform = np.concatenate([femur_l_result, fl_r, fl_a, fl_t, fl_pts])

In [ ]:
# rotate and shift origin to 0,0,0
femur_l_rotated = apply_rotate_zero_origin(femur_l_to_transform, femur_l_R)
femur_l_origin = femur_l_rotated[2] # femur head centre
femur_l_transformed = femur_l_rotated - femur_l_origin

In [ ]:
# scale surface for OpenSim and save
the_pts_transformed = femur_l_transformed[femur_l_lm_order['surf_pts']]/1000
new_mesh = pv.PolyData(the_pts_transformed, fl_faces)
new_mesh.save('model_update/tps_warping_results/femur_l.stl')

In [ ]:
# scale markers and muscles, create a dataframe for sorting and saving
body = 'femur_l'
femur_l_transformed_mrkr_mscl = {'body':[], 'name':[], 'location':[]}
for i, n in enumerate(femur_l_mrkrs_mscls_names):
    femur_l_transformed_mrkr_mscl['body'].append('femur_l'),
    femur_l_transformed_mrkr_mscl['name'].append(n),
    femur_l_transformed_mrkr_mscl['location'].append(femur_l_transformed[femur_l_lm_order['mrkrs_mscls']][i]/1000)
femur_l_transformed_mrkr_mscl_df = pd.DataFrame(femur_l_transformed_mrkr_mscl)

In [ ]:
fig = px.scatter()
fig.add_trace(go.Scatter3d(
x=femur_l_transformed[femur_l_lm_order['surf_pts'],0], 
y=femur_l_transformed[femur_l_lm_order['surf_pts'],1], 
z=femur_l_transformed[femur_l_lm_order['surf_pts'],2],
marker=dict(size=4, opacity=0.8), mode = 'markers'))

fig.add_trace(go.Scatter3d(
x=femur_r_transformed[femur_l_lm_order['surf_pts'],0], 
y=femur_r_transformed[femur_l_lm_order['surf_pts'],1], 
z=femur_r_transformed[femur_l_lm_order['surf_pts'],2],
marker=dict(size=4, opacity=0.8), mode = 'markers'))

fig.update_scenes(aspectmode="data" )
fig.show()

In [ ]:
# plot_transformed(femur_r_transformed, femur_r_lm_order, femur_r_mrkrs_mscls_names)
# plot_transformed(femur_l_transformed, femur_l_lm_order, femur_l_mrkrs_mscls_names)

#### *patella_r*

In [ ]:
patella_r_mrkrs_mscls_names[0]

In [ ]:
patella_r_mrkrs_mscls_names

In [ ]:
# rotation matrix
patella_r_location = np.array(mri_bone_markers['patella_r'])
points = ['patella_med_r', 'patella_r', 'patella_lat_r']

patella_r_pa = find_axis(mri_bone_markers, points)
patella_r_is = (np.array(mri_bone_markers['patella_sup_r']) - np.array(mri_bone_markers['patella_r']))/np.linalg.norm(np.array(mri_bone_markers['patella_sup_r']) - np.array(mri_bone_markers['patella_r']))
patella_r_lr = np.cross(patella_r_pa, patella_r_is)/np.linalg.norm(np.cross(patella_r_pa, patella_r_is))

patella_r_axes = np.array((patella_r_lr, patella_r_pa, patella_r_is)) * 50 + patella_r_location

patella_r_R, patella_r_t, patella_r_center_osim, patella_r_center_mri = axes_rotation(axes_osim, patella_r_axes)

In [ ]:
# assemble data for transformation and keep record of the order
obj_order = ['mrkrs_mscls','surf_pts']
patella_r_lm_order = {}
n = 0
for i, num in enumerate([patella_r_result.shape[0], ptlr_pts.shape[0]]):
    patella_r_lm_order[obj_order[i]] = list(range(n, n+num))
    n += num

patella_r_to_transform = np.concatenate([patella_r_result, ptlr_pts])

In [ ]:
# rotate and shift origin to 0,0,0
patella_r_rotated = apply_rotate_zero_origin(patella_r_to_transform, patella_r_R)
patella_r_origin = patella_r_rotated[0] # patella origin
patella_r_transformed = patella_r_rotated - patella_r_origin

In [ ]:
# scale surface for OpenSim and save
the_pts_transformed = patella_r_transformed[patella_r_lm_order['surf_pts']]/1000
new_mesh = pv.PolyData(the_pts_transformed, ptlr_faces)
new_mesh.save('model_update/tps_warping_results/patella_r.stl')

In [ ]:
# scale markers and muscles, create a dataframe for sorting and saving
body = 'patella_r'
patella_r_transformed_mrkr_mscl = {'body':[], 'name':[], 'location':[]}
for i, n in enumerate(patella_r_mrkrs_mscls_names):
    patella_r_transformed_mrkr_mscl['body'].append('patella_r'),
    patella_r_transformed_mrkr_mscl['name'].append(n),
    patella_r_transformed_mrkr_mscl['location'].append(patella_r_transformed[patella_r_lm_order['mrkrs_mscls']][i]/1000)
patella_r_transformed_mrkr_mscl_df = pd.DataFrame(patella_r_transformed_mrkr_mscl)

#### *patella_l*

In [ ]:
# rotation matrix
patella_l_location = np.array(mri_bone_markers['patella_l'])
points = ['patella_lat_l', 'patella_l', 'patella_med_l']

patella_l_pa = find_axis(mri_bone_markers, points)
patella_l_is = (np.array(mri_bone_markers['patella_sup_l']) - np.array(mri_bone_markers['patella_l']))/np.linalg.norm(np.array(mri_bone_markers['patella_sup_l']) - np.array(mri_bone_markers['patella_l']))
patella_l_lr = np.cross(patella_l_pa, patella_l_is)/np.linalg.norm(np.cross(patella_l_pa, patella_l_is))

patella_l_axes = np.array((patella_l_lr, patella_l_pa, patella_l_is)) * 50 + patella_l_location

patella_l_R, patella_l_t, patella_l_center_osim, patella_l_center_mri = axes_rotation(axes_osim, patella_l_axes)

In [ ]:
# assemble data for transformation and keep record of the order
obj_order = ['mrkrs_mscls','surf_pts']
patella_l_lm_order = {}
n = 0
for i, num in enumerate([patella_l_result.shape[0], ptll_pts.shape[0]]):
    patella_l_lm_order[obj_order[i]] = list(range(n, n+num))
    n += num

patella_l_to_transform = np.concatenate([patella_l_result, ptll_pts])

In [ ]:
# rotate and shift origin to 0,0,0
patella_l_rotated = apply_rotate_zero_origin(patella_l_to_transform, patella_l_R)
patella_l_origin = patella_l_rotated[0] # patella origin
patella_l_transformed = patella_l_rotated - patella_l_origin

In [ ]:
# scale surface for OpenSim and save
the_pts_transformed = patella_l_transformed[patella_l_lm_order['surf_pts']]/1000
new_mesh = pv.PolyData(the_pts_transformed, ptll_faces)
new_mesh.save('model_update/tps_warping_results/patella_l.stl')

In [ ]:
# scale markers and muscles, create a dataframe for sorting and saving
body = 'patella_l'
patella_l_transformed_mrkr_mscl = {'body':[], 'name':[], 'location':[]}
for i, n in enumerate(patella_l_mrkrs_mscls_names):
    patella_l_transformed_mrkr_mscl['body'].append('patella_r'),
    patella_l_transformed_mrkr_mscl['name'].append(n),
    patella_l_transformed_mrkr_mscl['location'].append(patella_l_transformed[patella_l_lm_order['mrkrs_mscls']][i]/1000)
patella_l_transformed_mrkr_mscl_df = pd.DataFrame(patella_l_transformed_mrkr_mscl)

In [ ]:
# fig = px.scatter()
# fig.add_trace(go.Scatter3d(
# x=patella_l_transformed[patella_l_lm_order['surf_pts'],0], 
# y=patella_l_transformed[patella_l_lm_order['surf_pts'],1], 
# z=patella_l_transformed[patella_l_lm_order['surf_pts'],2],
# marker=dict(size=4, opacity=0.8), mode = 'markers'))

# fig.add_trace(go.Scatter3d(
# x=patella_r_transformed[patella_l_lm_order['surf_pts'],0], 
# y=patella_r_transformed[patella_l_lm_order['surf_pts'],1], 
# z=patella_r_transformed[patella_l_lm_order['surf_pts'],2],
# marker=dict(size=4, opacity=0.8), mode = 'markers'))

# fig.update_scenes(aspectmode="data" )
# fig.show()

In [ ]:
plot_transformed(patella_r_transformed, patella_r_lm_order, patella_r_mrkrs_mscls_names)
plot_transformed(patella_l_transformed, patella_l_lm_order, patella_l_mrkrs_mscls_names)

#### *tibia_r*

In [ ]:
tibia_r_mrkrs_mscls_names[4]

In [ ]:
# rotation matrix
tibia_r_location = np.array(mri_bone_markers['knee_r_center'])

# points = [mri_bone_markers['knee_r_center'], mri_bone_markers['tibia_r_med'], mri_bone_markers['ankle_r_center'], mri_bone_markers['tibia_r_lat']]
# plane = Plane.best_fit(points)
# tibia_r_pa = np.array([plane.normal[0],plane.normal[1], plane.normal[2]])

points = ['tibia_r_med', 'ankle_r_center', 'tibia_r_lat']
tibia_r_pa = find_axis(mri_bone_markers, points)

tibia_r_is = (np.array(mri_bone_markers['knee_r_center']) - np.array(mri_bone_markers['ankle_r_center']))/np.linalg.norm(np.array(mri_bone_markers['ankle_r_center']) - np.array(mri_bone_markers['knee_r_center']))
tibia_r_lr = (np.cross(tibia_r_pa, tibia_r_is))/np.linalg.norm(np.cross(tibia_r_pa, tibia_r_is))

tibia_r_axes = np.array((tibia_r_lr, tibia_r_pa, tibia_r_is)) * 50 + tibia_r_location

tibia_r_R, tibia_r_t, tibia_r_center_osim, tibia_r_center_mri = axes_rotation(axes_osim, tibia_r_axes)

In [ ]:
# assemble data for transformation and keep record of the order
obj_order = ['mrkrs_mscls', 'wrp_rad', 'wrp_ax', 'wrp_transl', 'tib_surf_pts', 'fib_surf_pts']
tibia_r_lm_order = {}
n = 0
for i, num in enumerate([tibia_r_result.shape[0], tr_r.shape[0], tr_a.shape[0], tr_t.shape[0], tr_pts.shape[0], fib_r_pts.shape[0]]):
    tibia_r_lm_order[obj_order[i]] = list(range(n, n+num))
    n += num

tibia_r_to_transform = np.concatenate([tibia_r_result, tr_r, tr_a, tr_t, tr_pts, fib_r_pts])

In [ ]:
# rotate and shift origin to 0,0,0
tibia_r_rotated = apply_rotate_zero_origin(tibia_r_to_transform, tibia_r_R)
tibia_r_origin = tibia_r_rotated[4] # knee right centre
tibia_r_transformed = tibia_r_rotated - tibia_r_origin

In [ ]:
# scale surface for OpenSim and save
tr_pts_transformed = tibia_r_transformed[tibia_r_lm_order['tib_surf_pts']]/1000
tr_new_mesh = pv.PolyData(tr_pts_transformed, tr_faces)
tr_new_mesh.save('model_update/tps_warping_results/tibia_r.stl')

fr_pts_transformed = tibia_r_transformed[tibia_r_lm_order['fib_surf_pts']]/1000
fr_new_mesh = pv.PolyData(fr_pts_transformed, fib_r_faces )
fr_new_mesh.save('model_update/tps_warping_results/fibula_r.stl')

In [ ]:
# scale markers and muscles, create a dataframe for sorting and saving
body = 'tibia_r'
tibia_r_transformed_mrkr_mscl = {'body':[], 'name':[], 'location':[]}
for i, n in enumerate(tibia_r_mrkrs_mscls_names):
    tibia_r_transformed_mrkr_mscl['body'].append('tibia_r'),
    tibia_r_transformed_mrkr_mscl['name'].append(n),
    tibia_r_transformed_mrkr_mscl['location'].append(tibia_r_transformed[tibia_r_lm_order['mrkrs_mscls']][i]/1000)
tibia_r_transformed_mrkr_mscl_df = pd.DataFrame(tibia_r_transformed_mrkr_mscl)

#### *tibia_l*

In [ ]:
from skspatial.objects import Plane, Points

# rotation matrix
tibia_l_location = np.array(mri_bone_markers['knee_l_center'])

# points = [mri_bone_markers['knee_l_center'], mri_bone_markers['tibia_l_lat'], mri_bone_markers['ankle_l_center'], mri_bone_markers['tibia_l_med']]
# plane = Plane.best_fit(points) # 
# tibia_l_pa = np.array([plane.normal[0],plane.normal[1],plane.normal[2]])

points = ['tibia_l_lat', 'ankle_l_center', 'tibia_l_med']
tibia_l_pa = find_axis(mri_bone_markers, points)

tibia_l_is = (np.array(mri_bone_markers['knee_l_center'] - np.array(mri_bone_markers['ankle_l_center'])))/np.linalg.norm(np.array(mri_bone_markers['knee_l_center']) - np.array(mri_bone_markers['ankle_l_center']))
tibia_l_lr = (np.cross(tibia_l_pa, tibia_l_is))/np.linalg.norm(np.cross(tibia_l_pa, tibia_l_is))

tibia_l_axes = np.array((tibia_l_lr, tibia_l_pa, tibia_l_is)) * 50 + tibia_l_location

tibia_l_R, tibia_l_t, tibia_l_center_osim, tibia_l_center_mri = axes_rotation(axes_osim, tibia_l_axes)

In [ ]:
# assemble data for transformation and keep record of the order
obj_order = ['mrkrs_mscls', 'wrp_rad', 'wrp_ax', 'wrp_transl', 'tib_surf_pts', 'fib_surf_pts']
tibia_l_lm_order = {}
n = 0
for i, num in enumerate([tibia_l_result.shape[0], tl_r.shape[0], tl_a.shape[0], tl_t.shape[0], tl_pts.shape[0], fib_l_pts.shape[0]]):
    tibia_l_lm_order[obj_order[i]] = list(range(n, n+num))
    n += num

tibia_l_to_transform = np.concatenate([tibia_l_result, tl_r, tl_a, tl_t, tl_pts, fib_l_pts])

In [ ]:
# rotate and shift origin to 0,0,0
tibia_l_rotated = apply_rotate_zero_origin(tibia_l_to_transform, tibia_l_R)
tibia_l_origin = tibia_l_rotated[4] # knee right centre
tibia_l_transformed = tibia_l_rotated - tibia_l_origin

In [ ]:
# scale surface for OpenSim and save
tl_pts_transformed = tibia_l_transformed[tibia_l_lm_order['tib_surf_pts']]/1000
tl_new_mesh = pv.PolyData(tl_pts_transformed, tl_faces)
tl_new_mesh.save('model_update/tps_warping_results/tibia_l.stl')

fl_pts_transformed = tibia_l_transformed[tibia_l_lm_order['fib_surf_pts']]/1000
fl_new_mesh = pv.PolyData(fl_pts_transformed, fib_l_faces )
fl_new_mesh.save('model_update/tps_warping_results/fibula_l.stl')

In [ ]:
# fig = px.scatter()
# fig.add_trace(go.Scatter3d(
# x=tibia_l_transformed[tibia_l_lm_order['tib_surf_pts'],0], 
# y=tibia_l_transformed[tibia_l_lm_order['tib_surf_pts'],1], 
# z=tibia_l_transformed[tibia_l_lm_order['tib_surf_pts'],2],
# marker=dict(size=4, opacity=0.8), mode = 'markers'))

# fig.add_trace(go.Scatter3d(
# x=tibia_r_transformed[tibia_r_lm_order['tib_surf_pts'],0], 
# y=tibia_r_transformed[tibia_r_lm_order['tib_surf_pts'],1], 
# z=tibia_r_transformed[tibia_r_lm_order['tib_surf_pts'],2],
# marker=dict(size=4, opacity=0.8), mode = 'markers'))

# fig.update_scenes(aspectmode="data" )
# fig.show()

In [ ]:
plot_transformed(tibia_r_transformed, tibia_r_lm_order, tibia_r_mrkrs_mscls_names)

In [ ]:
plot_transformed(tibia_l_transformed, tibia_l_lm_order, tibia_l_mrkrs_mscls_names)

### Display transformed results

In [ ]:
#tibia_l_transformed
tibia_l_lm_order.keys()

In [ ]:
fig = px.scatter()

fig.add_trace(go.Scatter3d(
x=tibia_l_transformed[tibia_l_lm_order['mrkrs_mscls'],0], 
y=tibia_l_transformed[tibia_l_lm_order['mrkrs_mscls'],1], 
z=tibia_l_transformed[tibia_l_lm_order['mrkrs_mscls'],2],
marker=dict(
    size=4,
    opacity=0.8
),
mode = 'markers + text',
#text=tibia_l_mrkrs_mscls_names,
textposition="top center"))

# for i, num in enumerate(tibia_l_lm_order['wrp_rad']):
#     fig.add_trace(go.Scatter3d(
#     x=[tibia_l_transformed[tibia_l_lm_order['wrp_rad'][i],0], tibia_l_transformed[tibia_l_lm_order['wrp_transl'][i],0],tibia_l_transformed[tibia_l_lm_order['wrp_ax'][i],0]],
#     y=[tibia_l_transformed[tibia_l_lm_order['wrp_rad'][i],1],tibia_l_transformed[tibia_l_lm_order['wrp_transl'][i],1],tibia_l_transformed[tibia_l_lm_order['wrp_ax'][i],1]], 
#     z=[tibia_l_transformed[tibia_l_lm_order['wrp_rad'][i],2],tibia_l_transformed[tibia_l_lm_order['wrp_transl'][i],2],tibia_l_transformed[tibia_l_lm_order['wrp_ax'][i],2]],
# marker=dict(
#     size=4,
#     opacity=0.8
# )))

fig.add_trace(go.Scatter3d(
x=tibia_l_transformed[tibia_l_lm_order['wrp_rad'],0], 
y=tibia_l_transformed[tibia_l_lm_order['wrp_rad'],1], 
z=tibia_l_transformed[tibia_l_lm_order['wrp_rad'],2],
marker=dict(
    size=4,
    opacity=0.8
),
mode = 'markers'))

fig.add_trace(go.Scatter3d(
x=tibia_l_transformed[[0,1],0], 
y=tibia_l_transformed[[0,1],1], 
z=tibia_l_transformed[[0,1],2],
marker=dict(
    size=4,
    opacity=0.8
)))

fig.add_trace(go.Scatter3d(
x=[0,0], 
y=[0,0], 
z=[-35,35],
marker=dict(
    size=4,
    opacity=0.8
)))

fig.add_trace(go.Scatter3d(
x=[tibia_l_transformed[tibia_l_lm_order['wrp_rad'][5],0], tibia_l_transformed[tibia_l_lm_order['wrp_transl'][5],0],tibia_l_transformed[tibia_l_lm_order['wrp_ax'][5],0]],
y=[tibia_l_transformed[tibia_l_lm_order['wrp_rad'][5],1],tibia_l_transformed[tibia_l_lm_order['wrp_transl'][5],1],tibia_l_transformed[tibia_l_lm_order['wrp_ax'][5],1]], 
z=[tibia_l_transformed[tibia_l_lm_order['wrp_rad'][5],2],tibia_l_transformed[tibia_l_lm_order['wrp_transl'][5],2],tibia_l_transformed[tibia_l_lm_order['wrp_ax'][5],2]],
marker=dict(
size=4,
opacity=0.8
)))
fig.update_scenes(aspectmode="data" )
fig.show()

In [ ]:
# scale markers and muscles, create a dataframe for sorting and saving
body = 'tibia_l'
tibia_l_transformed_mrkr_mscl = {'body':[], 'name':[], 'location':[]}
for i, n in enumerate(tibia_l_mrkrs_mscls_names):
    tibia_l_transformed_mrkr_mscl['body'].append('tibia_l'),
    tibia_l_transformed_mrkr_mscl['name'].append(n),
    tibia_l_transformed_mrkr_mscl['location'].append(tibia_l_transformed[tibia_l_lm_order['mrkrs_mscls']][i]/1000)
tibia_l_transformed_mrkr_mscl_df = pd.DataFrame(tibia_l_transformed_mrkr_mscl)

In [ ]:
## collect all markers and muscles

In [ ]:
mrkrs_mscls_transformed_df = pd.concat([pelvis_transformed_mrkr_mscl_df, femur_r_transformed_mrkr_mscl_df, femur_l_transformed_mrkr_mscl_df, patella_r_transformed_mrkr_mscl_df, patella_l_transformed_mrkr_mscl_df, tibia_r_transformed_mrkr_mscl_df, tibia_l_transformed_mrkr_mscl_df])

In [ ]:
muscle_names = list(df_muscles['label'])
marker_names = list(df_markers['name'])

In [ ]:
list_of_rows = []
for i, row in mrkrs_mscls_transformed_df.iterrows():
    if row['name'] in muscle_names:
        list_of_rows.append(row)
mscls_transformed_df = pd.DataFrame(list_of_rows)
mscls_transformed_df.to_csv('model_update/tps_warping_results/muscles_transformed.csv', index=False)

In [ ]:
list_of_rows = []
for i, row in mrkrs_mscls_transformed_df.iterrows():
    if row['name'] in marker_names:
        list_of_rows.append(row)
markers_transformed_df = pd.DataFrame(list_of_rows)
markers_transformed_df.to_csv('model_update/tps_warping_results/markers_transformed.csv', index=False)

### Infer scaled parameters for muscle wrapping surfaces

In [ ]:
#[pelvis_wrp, femur_r_wrp, femur_l_wrp, tibia_r_wrp, tibia_l_wrp]

#### pelvis

In [ ]:
body = 'pelvis'
names_in_order = list(pelvis_wrp['name'])
transformed_landmarks = pelvis_transformed
landmark_order_dict = pelvis_lm_order
transformed_wrp_pelvis_df=return_transformed_wrapping_surfaces(transformed_landmarks, landmark_order_dict, names_in_order, body, mm_to_m = True)
# transformed_wrp_pelvis_d

In [ ]:
# wrp_df[wrp_df['body']=='pelvis'][['name', 'translation', 'rotation', 'half_length', 'radius']]

#### femur_l

In [ ]:
body = 'femur_l'
names_in_order = list(femur_l_wrp['name'])
transformed_landmarks = femur_l_transformed
landmark_order_dict = femur_l_lm_order
transformed_wrp_femur_l_df=return_transformed_wrapping_surfaces(transformed_landmarks, landmark_order_dict, names_in_order, body, mm_to_m = True)
# transformed_wrp_femur_l_df

In [ ]:
# wrp_df[wrp_df['body']=='femur_l'][['name', 'translation', 'rotation', 'half_length', 'radius']]

#### femur_r

In [ ]:
body = 'femur_r'
names_in_order = list(femur_r_wrp['name'])
transformed_landmarks = femur_r_transformed
landmark_order_dict = femur_r_lm_order
transformed_wrp_femur_r_df=return_transformed_wrapping_surfaces(transformed_landmarks, landmark_order_dict, names_in_order, body, mm_to_m = True)
# transformed_wrp_femur_r_df

In [ ]:
# wrp_df[wrp_df['body']=='femur_r'][['name', 'translation', 'rotation', 'half_length', 'radius']]

#### tibia_r

In [ ]:
body = 'tibia_r'
names_in_order = list(tibia_r_wrp['name'])
transformed_landmarks = tibia_r_transformed
landmark_order_dict = tibia_r_lm_order
transformed_wrp_tibia_r_df=return_transformed_wrapping_surfaces(transformed_landmarks, landmark_order_dict, names_in_order, body, mm_to_m = True)
#transformed_wrp_tibia_r_df

In [ ]:
# wrp_df[wrp_df['body']=='tibia_r'][['name', 'translation', 'rotation', 'half_length', 'radius']]

#### tibia_l

In [ ]:
body = 'tibia_l'
names_in_order = list(tibia_l_wrp['name'])
transformed_landmarks = tibia_l_transformed
landmark_order_dict = tibia_l_lm_order
transformed_wrp_tibia_l_df=return_transformed_wrapping_surfaces(transformed_landmarks, landmark_order_dict, names_in_order, body, mm_to_m = True)
# transformed_wrp_tibia_l_df

In [ ]:
# wrp_df[wrp_df['body']=='tibia_l'][['name', 'translation', 'rotation', 'half_length', 'radius']]

In [ ]:
wrp_transformed_df = pd.concat([transformed_wrp_pelvis_df, transformed_wrp_femur_r_df, transformed_wrp_femur_l_df,transformed_wrp_tibia_r_df,transformed_wrp_tibia_l_df])
wrp_transformed_df.to_csv('model_update/tps_warping_results/wrp_transformed.csv', index=False)

## Impute muscles to a scaled model
this does not guarantee the correct position of join centres. It only transfers muscles to the space of the 3DSlicer, where the muscles can be checked, corrected and then transformed together with bone markers to child body frame axes.